In [1]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.model_selection import cross_validate

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

import numpy as np
import os
import pandas as pd

# Data


In [2]:
data_path = "../data"
DATA = []
for file in os.listdir(data_path):
    DATA.append(pd.read_csv(os.path.join(data_path,file)))

In [3]:
for data in DATA:
    print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        5000 non-null   int64 
 1   job        5000 non-null   object
 2   marital    5000 non-null   object
 3   education  5000 non-null   object
 4   default    5000 non-null   object
 5   balance    5000 non-null   int64 
 6   housing    5000 non-null   object
 7   loan       5000 non-null   object
 8   contact    5000 non-null   object
 9   day        5000 non-null   int64 
 10  month      5000 non-null   object
 11  duration   5000 non-null   int64 
 12  campaign   5000 non-null   int64 
 13  pdays      5000 non-null   int64 
 14  previous   5000 non-null   int64 
 15  poutcome   5000 non-null   object
 16  y          5000 non-null   object
dtypes: int64(7), object(10)
memory usage: 664.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 

# Pipeline

In [4]:
num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer()),
    ('scale', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy="most_frequent")),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))
])


col_trans = ColumnTransformer([
    ('num_pipeline', num_pipeline, make_column_selector(dtype_include = np.number)),
    ('cat_pipeline', cat_pipeline, make_column_selector(dtype_include = np.object_))
])


In [5]:
classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    XGBClassifier(),
]

pipelines = []

In [6]:
for classifier in classifiers:
    pipelines.append((type(classifier), Pipeline([("transformer", col_trans), ("model", classifier)])))

In [7]:
pipelines

[(sklearn.tree._classes.DecisionTreeClassifier,
  Pipeline(steps=[('transformer',
                   ColumnTransformer(transformers=[('num_pipeline',
                                                    Pipeline(steps=[('impute',
                                                                     SimpleImputer()),
                                                                    ('scale',
                                                                     MinMaxScaler())]),
                                                    <sklearn.compose._column_transformer.make_column_selector object at 0x00000258CF2D4250>),
                                                   ('cat_pipeline',
                                                    Pipeline(steps=[('impute',
                                                                     SimpleImputer(strategy='most_frequent')),
                                                                    ('one-hot',
                                      

In [8]:
from sklearn.calibration import LabelEncoder


scores = []

for pipe in pipelines:
    pipe_score = []
    for data in DATA:
        score = cross_validate(pipe[1], data.iloc[:, :-1], LabelEncoder().fit_transform(data.iloc[:,-1]), cv = 5, scoring="accuracy")
        pipe_score.append(score["test_score"].mean())

    scores.append((pipe[0], pipe_score))

In [9]:
os.listdir("../data")

['banking_final.csv',
 'flights_final.csv',
 'mushrooms_final.csv',
 'weather_final.csv']

In [10]:
scores

[(sklearn.tree._classes.DecisionTreeClassifier,
  [0.8671999999999999, 0.913, 0.9795999999999999, 0.7896000000000001]),
 (sklearn.ensemble._forest.RandomForestClassifier,
  [0.8998000000000002, 0.9409999999999998, 0.9991999999999999, 0.8336]),
 (xgboost.sklearn.XGBClassifier,
  [0.9016, 0.9469999999999998, 0.9974000000000001, 0.8368])]

# Random Searching - searching for new defaults

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

param_distributions = [
    {
        "model__max_depth": randint(1,31),
        "model__min_samples_split":randint(2,61),
        "model__min_impurity_decrease":uniform(0,1),
        "model__min_samples_leaf":randint(1,61)
    },
    {
        "model__n_estimators":randint(1,2001),
        "model__min_samples_leaf":randint(1,5001),
        "model__max_samples":uniform(0.1,0.9),
        "model__max_features":uniform(0,1)
    },
    {
        "model__max_depth": randint(1,16),
        "model__min_child_weight": randint(1,129),
        "model__eta": uniform(2**(-10),1-2**(-10)),
        "model__alpha": uniform(2**(-10),2**(10))
    }
]

best_params = [[],[],[],[]]
pipe_best_models = []
pipe_best_scores = []
history = [[],[],[]]
for i,pipe in enumerate(pipelines):    
    for j,data in enumerate(DATA):
        rs = RandomizedSearchCV(pipe[1], 
                                param_distributions= param_distributions[i],
                                verbose=True,
                                random_state=42,
                                cv=5,
                                n_iter=200,
                                n_jobs=-1,
                                scoring="roc_auc"
                                )
        rs.fit(data.iloc[:, :-1],LabelEncoder().fit_transform(data.iloc[:,-1]))
        pipe_best_scores.append(rs.best_score_)
        pipe_best_models.append(rs.best_estimator_)
        best_params[j].append(rs.best_params_)
        history[i].append(rs.cv_results_)    


Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits


c:\Users\macie\OneDrive - Politechnika Warszawska\Pulpit\STUDIA\SEMESTR5\AutoML\Projekty\AutoML-project1\automl1\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [12]:
history_datasets = []
for h in history:
    df = pd.concat([pd.DataFrame(h[i]) for i in range(len(h))], keys=range(len(h)), names=['dataset'])
    df = df.reset_index()
    df.drop(columns='level_1', inplace=True)
    history_datasets.append(df)
print(f"{pipelines[0][0]} shape: {history_datasets[0].shape}")
print(f"{pipelines[1][0]} shape: {history_datasets[1].shape}")
print(f"{pipelines[2][0]} shape: {history_datasets[2].shape}")

<class 'sklearn.tree._classes.DecisionTreeClassifier'> shape: (800, 18)
<class 'sklearn.ensemble._forest.RandomForestClassifier'> shape: (800, 18)
<class 'xgboost.sklearn.XGBClassifier'> shape: (800, 18)


In [13]:
model_names = ['DecisionTree','RandomForest','XGBoost']

# Saving history to csv files

In [14]:
for i, df in enumerate(history_datasets):
    df.to_csv(f'../history/history_dataset_{model_names[i]}.csv', index=False)

# Reading history from csv

In [15]:
history_DecisionTree = pd.read_csv('../history/history_dataset_DecisionTree.csv')
history_RandomForest = pd.read_csv('../history/history_dataset_RandomForest.csv')
history_XGBoost = pd.read_csv('../history/history_dataset_XGBoost.csv')

In [16]:
history_DecisionTree.head()

,dataset,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,param_model__min_impurity_decrease,param_model__min_samples_leaf,param_model__min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0,0.045820,0.002683,0.014951,0.001641,7,0.796543,15,44,"{'model__max_depth': 7, 'model__min_impurity_d...",0.5,0.5,0.5,0.5,0.5,0.5,0.0,7
1,0,0.049378,0.009177,0.016595,0.008528,8,0.598658,39,59,"{'model__max_depth': 8, 'model__min_impurity_d...",0.5,0.5,0.5,0.5,0.5,0.5,0.0,7
2,0,0.058931,0.009848,0.009868,0.002243,19,0.099975,11,25,"{'model__max_depth': 19, 'model__min_impurity_...",0.5,0.5,0.5,0.5,0.5,0.5,0.0,7
3,0,0.047674,0.013412,0.012412,0.003954,21,0.601115,24,4,"{'model__max_depth': 21, 'model__min_impurity_...",0.5,0.5,0.5,0.5,0.5,0.5,0.0,7
4,0,0.048580,0.012482,0.013218,0.004257,22,0.056412,24,45,"{'model__max_depth': 22, 'model__min_impurity_...",0.5,0.5,0.5,0.5,0.5,0.5,0.0,7


In [17]:
history_DecisionTree.shape

(800, 18)

In [18]:
history_RandomForest.head()

,dataset,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_features,param_model__max_samples,param_model__min_samples_leaf,param_model__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0,2.430337,0.085638,0.093527,0.002885,0.374540,0.955643,3773,1045,"{'model__max_features': 0.3745401188473625, 'm...",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,25
1,0,3.906768,0.849372,0.148279,0.031769,0.156019,0.240395,4427,1483,"{'model__max_features': 0.15601864044243652, '...",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,25
2,0,16.546819,2.208717,0.250995,0.033240,0.866176,0.641004,131,1686,"{'model__max_features': 0.8661761457749352, 'm...",0.880381,0.867381,0.873324,0.833648,0.860107,0.862968,0.016111,2
3,0,3.203614,0.641014,0.135754,0.049218,0.056412,0.749799,2434,1216,"{'model__max_features': 0.056411579027100256, ...",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,25
4,0,0.095550,0.007481,0.016689,0.003296,0.992212,0.655733,3386,22,"{'model__max_features': 0.9922115592912175, 'm...",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,25


In [19]:
history_XGBoost.head()

,dataset,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,param_model__eta,param_model__max_depth,param_model__min_child_weight,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0,0.120718,0.009384,0.015755,0.004705,383.530058,0.950762,11,72,"{'model__alpha': 383.5300582621992, 'model__et...",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,72
1,0,0.130825,0.014063,0.021409,0.013045,613.027264,0.156843,3,87,"{'model__alpha': 613.0272643802655, 'model__et...",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,72
2,0,0.158108,0.035043,0.017295,0.002255,59.478595,0.866307,4,104,"{'model__alpha': 59.47859542273625, 'model__et...",0.840699,0.839393,0.830057,0.802664,0.842297,0.831022,0.014805,19
3,0,0.117092,0.014543,0.014069,0.002245,725.067296,0.021541,2,88,"{'model__alpha': 725.0672962256506, 'model__et...",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,72
4,0,0.113698,0.008851,0.015426,0.002006,852.422241,0.213108,12,21,"{'model__alpha': 852.4222407421319, 'model__et...",0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,72


# New defaults below

In [20]:
def get_best_params_overall(df):
    df['params_str'] = df['params'].apply(lambda x: str(x))
    grouped_mean = df.groupby(['params_str'])['mean_test_score'].mean().reset_index()
    grouped_mean.sort_values(by='mean_test_score', ascending=False, inplace=True)
    return grouped_mean.iloc[0, 0], grouped_mean.iloc[0, 1]

In [21]:
best_params_DecisionTree, best_params_DecisionTree_score = get_best_params_overall(history_DecisionTree)
print(f"Best params for DecisionTree: {best_params_DecisionTree}") 
print(f"with score: {best_params_DecisionTree_score}")

Best params for DecisionTree: {'model__max_depth': 25, 'model__min_impurity_decrease': 0.005061583846218687, 'model__min_samples_leaf': 46, 'model__min_samples_split': 19}
with score: 0.829105606507877


In [22]:
best_params_RandomForest, best_params_RandomForest_score = get_best_params_overall(history_RandomForest)
print(f"Best params for RandomForest: {best_params_RandomForest}") 
print(f"with score: {best_params_RandomForest_score}")

Best params for RandomForest: {'model__max_features': 0.02431596643145384, 'model__max_samples': 0.680925066316451, 'model__min_samples_leaf': 17, 'model__n_estimators': 1196}
with score: 0.8965365914913082


In [23]:
best_params_XGBoost, best_params_XGBoost_score = get_best_params_overall(history_XGBoost)
print(f"Best params for XGBoost: {best_params_XGBoost}")
print(f"With score: {best_params_XGBoost_score}")

Best params for XGBoost: {'model__alpha': 14.418715022746483, 'model__eta': 0.1996247845535622, 'model__max_depth': 8, 'model__min_child_weight': 35}
With score: 0.9250081472612419


# Tunability

##### Now let's compute tunability of each of the ML algorithms. We'll start with looking for the optimal configuration of the hyperparameters for each of the datasets

In [24]:
def get_best_params_per_dataset(df):
    df['params_str'] = df['params'].apply(lambda x: str(x))
    best_params_per_dataset = df.sort_values(['dataset', 'rank_test_score'], ascending=[True, True]).groupby('dataset').first().reset_index()
    best_params_per_dataset.rename(columns={'params_str': 'best_params', 'mean_test_score': 'best_score'}, inplace=True)
    best_params_per_dataset = best_params_per_dataset[['dataset', 'best_params', 'best_score']]
    default_params, _ = get_best_params_overall(df)
    score_for_default_params = df[df['params_str'] == default_params][['dataset', 'mean_test_score']].rename(columns={'mean_test_score': 'default_score'})
    best_params_per_dataset = best_params_per_dataset.merge(score_for_default_params, on='dataset', how='left')
    best_params_per_dataset['abs_tunability'] = best_params_per_dataset['best_score'] - best_params_per_dataset['default_score']
    best_params_per_dataset['rel_tunability (%)'] = best_params_per_dataset['abs_tunability'] / best_params_per_dataset['default_score'] * 100 
    return best_params_per_dataset

In [25]:
best_params_per_dataset_DecisionTree = get_best_params_per_dataset(history_DecisionTree)
best_params_per_dataset_DecisionTree

,dataset,best_params,best_score,default_score,abs_tunability,rel_tunability (%)
0,0,"{'model__max_depth': 25, 'model__min_impurity_...",0.775184,0.775184,0.0,0.0
1,1,"{'model__max_depth': 25, 'model__min_impurity_...",0.934547,0.934547,0.0,0.0
2,2,"{'model__max_depth': 25, 'model__min_impurity_...",0.855210,0.855210,0.0,0.0
3,3,"{'model__max_depth': 25, 'model__min_impurity_...",0.751481,0.751481,0.0,0.0


In [26]:
best_params_per_dataset_RandomForest = get_best_params_per_dataset(history_RandomForest)
best_params_per_dataset_RandomForest

,dataset,best_params,best_score,default_score,abs_tunability,rel_tunability (%)
0,0,"{'model__max_features': 0.02431596643145384, '...",0.866173,0.866173,0.000000,0.000000
1,1,"{'model__max_features': 0.02431596643145384, '...",0.957959,0.957959,0.000000,0.000000
2,2,"{'model__max_features': 0.02431596643145384, '...",0.938172,0.938172,0.000000,0.000000
3,3,"{'model__max_features': 0.8661761457749352, 'm...",0.828975,0.823842,0.005133,0.623063


In [27]:
best_params_per_dataset_XGBoost = get_best_params_per_dataset(history_XGBoost)
best_params_per_dataset_XGBoost

,dataset,best_params,best_score,default_score,abs_tunability,rel_tunability (%)
0,0,"{'model__alpha': 14.418715022746483, 'model__e...",0.892356,0.892356,0.000000,0.000000
1,1,"{'model__alpha': 14.418715022746483, 'model__e...",0.977244,0.977244,0.000000,0.000000
2,2,"{'model__alpha': 14.418715022746483, 'model__e...",0.984947,0.984947,0.000000,0.000000
3,3,"{'model__alpha': 14.73990891937001, 'model__et...",0.845808,0.845485,0.000323,0.038144
